# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
import requests
import os

## Read and process static data

### Paths

In [2]:
path_npvm_zones = os.path.join("..", "data", "Verkehrszonen_Schweiz_NPVM_2017_shp.zip")
path_mobility_stations = os.path.join("..", "data", "mobility-stationen-und-fahrzeuge-schweiz.csv")
path_pt_jrta = os.path.join("..", "data", "140_JRTA_(OEV).mtx")
path_pt_ntr = os.path.join("..", "data", "144_NTR_(OEV).mtx")

### Read NPVM-zones with shapes

In [3]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 4.23 s
Wall time: 4.25 s


In [4]:
gdf_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

Anzahl NPVM-Zonen: 7978


In [6]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [7]:
get_npvm_zone(5301003)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
84,5301003,53,53,Bülach,1,None,1,ZH,1,Städtisch,261,Zürich,12032,Kloten,"POLYGON ((8.53973 47.51618, 8.54007 47.51529, ..."


### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [8]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [9]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

Anzahl Mobility Fahrzeuge: 2662


In [10]:
df_mobility_stations = df_mobility_vechicles.groupby("Stationsnummer").first().reset_index()

In [11]:
df_mobility_stations["lon"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_stations["lat"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[0])
df_mobility_stations = gpd.GeoDataFrame(df_mobility_stations, geometry=gpd.points_from_xy(df_mobility_stations.lon, df_mobility_stations.lat), crs=4326)

In [12]:
print("Anzahl Mobility Stationen: {}".format(len(df_mobility_stations)))

Anzahl Mobility Stationen: 1550


In [13]:
gdf_mobilty_stations_with_npvm_zone = gpd.sjoin(df_mobility_stations, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [14]:
print("Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_stations_with_npvm_zone)))

Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: 1549


In [15]:
gdf_mobilty_stations_with_npvm_zone.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,1006,Brugg Bahnhof,POINT (8.20942 47.48154),409501008,Brugg
719,3215,Brugg Post Neumarkt / Bahnhofstrasse,POINT (8.20757 47.48216),409501008,Brugg
1,1012,Arbon Bahnhof,POINT (9.43345 47.51032),440101010,Arbon
2,1019,Basel Vogesenstrasse,POINT (7.57483 47.56869),270101028,Basel
3,1024,Bellinzona Stazione,POINT (9.03017 46.19630),500201015,Bellinzona


In [16]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_stations_with_npvm_zone.dissolve(by="ID", aggfunc={"N_Gem": "first", "Name": lambda x: list(x), "Stationsnummer": lambda x: list(x)}).reset_index()

In [17]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Standort: 1311


In [18]:
"{}".format(gdf_npvm_zones_with_mobility_station[gdf_npvm_zones_with_mobility_station.N_Gem == "Samedan"])

'            ID                  geometry    N_Gem  \\\n911  378601001  POINT (9.87270 46.53347)  Samedan   \n\n                               Name Stationsnummer  \n911  [Samedan Bahnhof / via Retica]         [4906]  '

### Read PT-skims

In [19]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

CPU times: total: 8.53 s
Wall time: 8.56 s


In [20]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

CPU times: total: 8.33 s
Wall time: 8.38 s


In [21]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [22]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [23]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [24]:
get_jrta(378601001, 35101026)

286.0875843780647

In [25]:
get_ntr(378601001, 35101026)

3.000000000000001

## Execute query

### Define origin and destination and assign NPVM-Zone

In [26]:
orig_lon_lat = (7.423570, 46.936620)
dest_lon_lat = (7.695260, 46.828541)

In [27]:
def get_npvm_zone_id(point_easting_northing, gdf_npvm_zones):
    point = Point(point_easting_northing[0], point_easting_northing[1])
    gdf_point = gpd.GeoDataFrame({'geometry': [point]}, crs="EPSG:4326")
    gdf_point_with_zone = gpd.sjoin(gdf_point, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
    return gdf_point_with_zone["ID"].item()

In [28]:
get_npvm_zone_id((7.423570, 46.936620), gdf_npvm_zones)

35101026

In [29]:
orig_zone_id = get_npvm_zone_id(orig_lon_lat, gdf_npvm_zones)
dest_zone_id = get_npvm_zone_id(dest_lon_lat, gdf_npvm_zones)

In [30]:
print(orig_zone_id)
print(dest_zone_id)
get_jrta(orig_zone_id, dest_zone_id)

35101026
92301001


101.2076327070268

In [78]:
def get_potential_mobility_stations(orig_zone_id, dest_zone_id, gdf_mobilty_stations_with_npvm_zone, factor=1.5, constant=30.0):
    jrta_orig_dest = get_jrta(orig_zone_id, dest_zone_id)
    potential_stations_ids = []
    for station_id, zone_id in gdf_mobilty_stations_with_npvm_zone[["Stationsnummer", "ID"]].values.tolist():
        jrta_orig_station = get_jrta(orig_zone_id, zone_id)
        jrta_station_dest = get_jrta(zone_id, dest_zone_id)
        if jrta_orig_station + jrta_station_dest <= factor * jrta_orig_dest + constant:
            potential_stations_ids += [station_id]
    df_potential_station_ids = pd.DataFrame(potential_stations_ids, columns=["Stationsnummer"])
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_potential_station_ids, on=["Stationsnummer"])

In [79]:
get_potential_mobility_stations(orig_zone_id, dest_zone_id, gdf_mobilty_stations_with_npvm_zone)

,Stationsnummer,Name,geometry,ID,N_Gem
0,1027,Belp Bahnhof,POINT (7.50005 46.88650),86101006,Belp
1,1040,Burgdorf Bahnhof,POINT (7.61934 47.06081),40401007,Burgdorf
2,1041,Burgdorf Steinhof,POINT (7.61717 47.05301),40401009,Burgdorf
3,4402,Burgdorf Pestalozzistrasse / Berner Fachhochsc...,POINT (7.62025 47.05750),40401009,Burgdorf
4,1075,Köniz Bahnhof,POINT (7.41546 46.92504),35501018,Köniz
...,...,...,...,...,...
193,5908,Burgdorf Oberstadt / Emmentalstrasse,POINT (7.62482 47.05484),40401014,Burgdorf
194,5909,Bern Kollerweg,POINT (7.45487 46.94371),35101082,Bern
195,5926,Grosshöchstetten Bahnhof,POINT (7.63551 46.90527),60801001,Grosshöchstetten
196,5963,Bern Huebergass,POINT (7.41610 46.94553),35101025,Bern


In [33]:
orig_point = Point(orig_lon_lat[0], orig_lon_lat[1])
dest_point = Point(dest_lon_lat[0], dest_lon_lat[1])

In [34]:
gdf_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
gdf_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [35]:
gdf_orig_with_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
gdf_dest_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]

In [36]:
gdf_orig_with_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [37]:
gdf_dest_zone

,ID,N_Gem,geometry
0,35101026,Bern,POINT (7.42357 46.93662)


In [38]:
orig_zone_id = gdf_orig_with_zone["ID"].item()

### Compute potential NPVM-zones with Mobility-station

In [39]:
gdf_circle_around_orig  = gpd.GeoDataFrame(geometry=gdf_orig_with_zone.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
gdf_circle_around_dest = gpd.GeoDataFrame(geometry=gdf_dest_zone.to_crs(2026).buffer(100*1000).to_crs(4326), crs="EPSG:4326")

In [40]:
gdf_circle_around_orig

,geometry
0,"POLYGON ((7.40836 46.90462, 7.40386 46.90580, ..."


In [41]:
gdf_circle_around_dest

,geometry
0,"POLYGON ((7.12418 46.29940, 7.03584 46.32239, ..."


In [42]:
gdf_npvm_zones_with_mobility_station_in_orig_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_orig)
gdf_npvm_zones_with_mobility_station_in_dest_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_dest)

In [43]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_orig_circle)))
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_dest_circle)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: 63
Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: 549


In [44]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,Name,Stationsnummer,index_right
347,35101004,"MULTIPOINT (7.38055 46.94513, 7.38364 46.94701)",Bern,"[Bern Migros Bethlehem, Bern Brünnen / Riedbac...","[2180, 4718]",0
348,35101006,POINT (7.38535 46.93170),Bern,[Bern Bümpliz / Wangenstrasse],[5248],0
349,35101007,POINT (7.38609 46.94300),Bern,[Bern Bümpliz / Brünnenstrasse],[4604],0
350,35101011,POINT (7.38905 46.94569),Bern,[Bern Bümpliz Nord Bahnhof],[2902],0
351,35101016,"MULTIPOINT (7.39396 46.94112, 7.39438 46.93700...",Bern,"[Bern Bümpliz Süd Bahnhof, Bern Bümpliz Zentru...","[2537, 3921, 4073]",0


In [45]:
gdf_npvm_zones_with_mobility_station_in_dest_circle[gdf_npvm_zones_with_mobility_station_in_dest_circle.N_Gem == "Samedan"]

,ID,geometry,N_Gem,Name,Stationsnummer,index_right


In [46]:
gdf_npvm_zones_with_potential_mobility_station = pd.concat([gdf_npvm_zones_with_mobility_station_in_orig_circle, gdf_npvm_zones_with_mobility_station_in_dest_circle]).groupby("ID").first().reset_index().drop(["index_right"], axis=1)

In [47]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: {}".format(len(gdf_npvm_zones_with_potential_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: 549


In [48]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem,Name,Stationsnummer
0,30101004,POINT (7.27780 47.04357),Aarberg,[Aarberg Bahnhof],[2672]
1,30601005,POINT (7.30569 47.07726),Lyss,[Lyss Bahnhof],[1579]
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565]
3,30601012,POINT (7.31873 47.09890),Lyss,[Busswil Bahnhof],[4222]
4,31101002,POINT (7.38656 47.04189),Schüpfen,[Schüpfen Bahnhof],[1249]


In [49]:
gdf_npvm_zones_with_potential_mobility_station[gdf_npvm_zones_with_potential_mobility_station.ID == 30601007]

,ID,geometry,N_Gem,Name,Stationsnummer
2,30601007,POINT (7.30560 47.07900),Lyss,[Lyss Busswilstrasse],[4565]


### Get Mobility travel time and distance from ORMS

In [50]:
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

In [73]:
# TODO hier weitermachen. Diese Funktion funktioniert noch nicht wie gewünscht
def collect_data_on_potential_npvm_zones(orig_zone_id, gdf_npvm_zones_with_potential_mobility_station):
    list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())
    coords_str = "{},{}".format(dest_point.x, dest_point.y)
    for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
        center = pot_mob_st[3].centroid
        coords_str += ";{},{}".format(center.x, center.y)
    url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)
    res = requests.get(url).json()
    road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[4]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
    road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[4]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
    
    pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
    pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
    print(len(pd_distances))
    
    pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
    pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0
    print(len(pd_durations))

    gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_distances, on=["ID"])
    gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_durations, on=["ID"])
    
    print(len(gdf_npvm_zones_with_potential_mobility_station_with_data))
    
    zone_ids_list = [x.item() for x in gdf_npvm_zones_with_potential_mobility_station[["ID"]].values]
    jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
    ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]
    
    pd_jrtas = pd.DataFrame(jrta_list, columns=["ID", OEV_JRTA_VON_START_MIN])
    pd_ntrs = pd.DataFrame(ntr_list, columns=["ID", OEV_NTR_VON_START])
    
    gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_jrtas, on=["ID"])
    gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_ntrs, on=["ID"])
    
    return gdf_npvm_zones_with_potential_mobility_station_with_data

In [74]:
gdf_npvm_zones_with_potential_mobility_station = get_potential_mobility_stations(orig_zone_id, dest_zone_id, gdf_mobilty_stations_with_npvm_zone)
gdf_npvm_zones_with_potential_mobility_station.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,1027,Belp Bahnhof,POINT (7.50005 46.88650),86101006,Belp
1,1040,Burgdorf Bahnhof,POINT (7.61934 47.06081),40401007,Burgdorf
2,1041,Burgdorf Steinhof,POINT (7.61717 47.05301),40401009,Burgdorf
3,4402,Burgdorf Pestalozzistrasse / Berner Fachhochsc...,POINT (7.62025 47.05750),40401009,Burgdorf
4,1075,Köniz Bahnhof,POINT (7.41546 46.92504),35501018,Köniz


In [75]:
collect_data_on_potential_npvm_zones(orig_zone_id, gdf_npvm_zones_with_potential_mobility_station)

148
148
198


,Stationsnummer,Name,geometry,ID,N_Gem,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start
0,1027,Belp Bahnhof,POINT (7.50005 46.88650),86101006,Belp,30.0800,29.845000,30.413020,0.000000
1,1040,Burgdorf Bahnhof,POINT (7.61934 47.06081),40401007,Burgdorf,42.7260,51.641667,45.149267,1.000000
2,1041,Burgdorf Steinhof,POINT (7.61717 47.05301),40401009,Burgdorf,41.2089,48.831667,52.161444,1.565170
3,1041,Burgdorf Steinhof,POINT (7.61717 47.05301),40401009,Burgdorf,41.2089,48.831667,52.161444,1.565170
4,1041,Burgdorf Steinhof,POINT (7.61717 47.05301),40401009,Burgdorf,41.2089,48.831667,52.161444,1.565170
...,...,...,...,...,...,...,...,...,...
715,5908,Burgdorf Oberstadt / Emmentalstrasse,POINT (7.62482 47.05484),40401014,Burgdorf,40.7884,47.708333,45.399546,1.000000
716,5909,Bern Kollerweg,POINT (7.45487 46.94371),35101082,Bern,36.6293,33.620000,25.576906,1.020846
717,5926,Grosshöchstetten Bahnhof,POINT (7.63551 46.90527),60801001,Grosshöchstetten,18.9012,22.728333,66.197192,2.000207
718,5963,Bern Huebergass,POINT (7.41610 46.94553),35101025,Bern,39.9269,38.528333,15.428683,1.000000


In [ ]:
list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())

In [ ]:
list_npvm_zones_with_potential_mobility_station[:3]

In [ ]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
    center = pot_mob_st[3].centroid
    coords_str += ";{},{}".format(center.x, center.y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [ ]:
%time res = requests.get(url).json()

In [ ]:
road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}

In [ ]:
print("Anzahl Distanzen: {}".format(len(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))
print("Anzahl Reisezeiten: {}".format(len(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))

In [ ]:
gdf_npvm_zones_with_potential_mobility_station.head()

In [ ]:
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

In [ ]:
pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0

gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_distances, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_durations, on=["ID"])


In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data.head()

### Get pt skims from origin to potential Mobility stations

In [ ]:
OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

In [ ]:
zone_ids_list = [x.item() for x in gdf_npvm_zones_with_potential_mobility_station[["ID"]].values]
jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]

In [ ]:
pd_jrtas = pd.DataFrame(jrta_list, columns=["ID", OEV_JRTA_VON_START_MIN])
pd_ntrs = pd.DataFrame(ntr_list, columns=["ID", OEV_NTR_VON_START])

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_jrtas, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_ntrs, on=["ID"])

In [ ]:
print(len(gdf_npvm_zones_with_potential_mobility_station_with_data))

In [ ]:
KOSTEN_CHF = "Kosten_CHF"

In [ ]:
CHF_PER_KM_MOBILITY = 0.75
MIN_PER_TRANSFER = 20.0
FILTER_FACTOR = 1.05

In [ ]:
def calc_costs(vtt_chf_per_h=20):
    gdf_npvm_zones_with_potential_mobility_station_with_data[KOSTEN_CHF] = \
        CHF_PER_KM_MOBILITY * gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_DISTANZ_BIS_ZIEL_KM] + \
        (gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_ZEIT_BIS_ZIEL_MIN] + gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_JRTA_VON_START_MIN] + 
        gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_NTR_VON_START] * MIN_PER_TRANSFER) / 60.0 * vtt_chf_per_h
    return gdf_npvm_zones_with_potential_mobility_station_with_data.sort_values(by=KOSTEN_CHF, ascending=True)

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data = calc_costs(20)

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Samedan"]

In [ ]:
def get_best_mobility_stations_per_vtt(vtt_chf_per_h):
    df_tmp = calc_costs(vtt_chf_per_h=vtt_chf_per_h)
    min_cost = df_tmp[KOSTEN_CHF].min()
    df_tmp = df_tmp[df_tmp[KOSTEN_CHF] <= min_cost * FILTER_FACTOR]
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_tmp[["ID", KOSTEN_CHF]], on="ID").sort_values(by=KOSTEN_CHF)

In [ ]:
get_best_mobility_stations_per_vtt(0)

In [ ]:
best_mobility_stations_per_vtt = {vtt: get_best_mobility_stations_per_vtt(vtt) for vtt in range(0, 101)}

In [ ]:
best_mobility_stations_per_vtt[57]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Steffisburg"]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Thun"]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Bern"].head(50)

### Visualize situation on map

In [ ]:
bounds = gdf_npvm_zones_with_potential_mobility_station_with_data.total_bounds
bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
bounds

In [ ]:
best_mobility_stations_per_vtt[11]

In [ ]:
def show_best_mobility_stations_per_vtt(vtt):
    geo_data = GeoData(geo_dataframe=best_mobility_stations_per_vtt[vtt],
        style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Mobility-Stationen')
    for l in m.layers:
        if type(l) == GeoData:
            m.remove_layer(l)
    m.add_layer(geo_data)

In [ ]:
def on_slider_changed(event):
    show_best_mobility_stations_per_vtt(event["new"])

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, Marker, WidgetControl
from ipywidgets import IntSlider
import geopandas
import json

vtt_slider = IntSlider(description='Zeitkosten', min=0, max=100, value=20)
vtt_slider.observe(on_slider_changed, names='value')

widget_control = WidgetControl(widget=vtt_slider, position='topright')

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap, scroll_wheel_zoom=True)
m.add_control(widget_control)

m.fit_bounds(bounds)
show_best_mobility_stations_per_vtt(vtt_slider.value)
m.add_control(FullScreenControl())

m.layout.width = '100%'
m.layout.height = '500px'

orig_marker = Marker(location=(orig_lon_lat[1], orig_lon_lat[0]) , draggable=False)
dest_marker = Marker(location=(dest_lon_lat[1], dest_lon_lat[0]) , draggable=False)

m.add_layer(orig_marker)
m.add_layer(dest_marker)

m